In [345]:
# En primer lugar se instalan e importan las librerías y dependencias que serán utilizadas en el proceso.
!pip install SPARQLWrapper  
!pip install wikipedia      
import pandas as pd
from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import wikipedia
import json
import xml.etree.ElementTree as ET

In [346]:
# A continuación se pregunta al usuario por el identificador de la clase del cual se quiere generar un diagrama
print("¿Cual es el identificador de la clase del cual se quiere generar un diagrama?")
clase = input()

¿Cual es el identificador de la clase del cual se quiere generar un diagrama?
Q1004


In [347]:
#Se define la función para consultar Wikidata. Se consulta el endpoint señalado y 
#devuelve los resultados como un DataFrame de Pandas.

def query_wikidata(sparql_query, sparql_service_url):        
  
    # crear la conexión con el endpoint
    sparql = SPARQLWrapper(sparql_service_url)    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # solicitar resultados
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

#Se genera la consulta para obtener el numero de instancias de la clase a representar
sparql_query1 = """SELECT DISTINCT ?clase ?claseLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""".
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}
"""
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
instancias_wikidata = query_wikidata(sparql_query1, sparql_service_url)

In [348]:
#Se confirma que la consulta haya salido bien
instancias_wikidata.head()

,clase.type,clase.value,claseLabel.type,claseLabel.value,claseLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q91879,literal,Q91879,NaN
1,uri,http://www.wikidata.org/entity/Q91892,literal,Q91892,NaN
2,uri,http://www.wikidata.org/entity/Q91912,literal,Q91912,NaN
3,uri,http://www.wikidata.org/entity/Q91943,literal,Q91943,NaN
4,uri,http://www.wikidata.org/entity/Q92191,literal,Q92191,NaN


In [349]:
#Se obtiene el numero de instancias de la clase de entrada
numero_de_instancias = instancias_wikidata.shape[0]
numero_de_instancias

4589

In [350]:
#Se genera la consulta para obtener las propiedades de la clase de entrada
sparql_query2 = """SELECT DISTINCT ?p ?pLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""";
?p ?x.
#to obtain property labels:
?property wikibase:directClaim ?p
#to obtain labels in spanish:
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}   
    """
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
propiedades_wikidata = query_wikidata(sparql_query2, sparql_service_url)

In [351]:
#Se confirma que la consulta haya salido bien
propiedades_wikidata.head()

,p.type,p.value,pLabel.type,pLabel.value
0,uri,http://www.wikidata.org/prop/direct/P279,literal,http://www.wikidata.org/prop/direct/P279
1,uri,http://www.wikidata.org/prop/direct/P407,literal,http://www.wikidata.org/prop/direct/P407
2,uri,http://www.wikidata.org/prop/direct/P495,literal,http://www.wikidata.org/prop/direct/P495
3,uri,http://www.wikidata.org/prop/direct/P580,literal,http://www.wikidata.org/prop/direct/P580
4,uri,http://www.wikidata.org/prop/direct/P577,literal,http://www.wikidata.org/prop/direct/P577


In [352]:
#Se genera la consulta para obtener las propiedades de la clase.
sparql_query2 = """SELECT DISTINCT ?p ?pLabel
WHERE {
?clase wdt:P31 wd:"""+clase+""";
?p ?x.
#to obtain property labels:
?property wikibase:directClaim ?p
#to obtain labels in spanish:
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}   
    """
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
propiedades_wikidata = query_wikidata(sparql_query2, sparql_service_url)

#Se listan las propiedades de salida de la clase y se le asignan a la variable "lista_propiedades"
lista_completa = propiedades_wikidata["p.value"].tolist()
lista_propiedades = []
for ele in lista_completa:
    ele = ele[36:]
    lista_propiedades.append(ele)

In [ ]:
#Para cada propiedad de la lista:
for prop in lista_propiedades:
    
    #Se obtienen de las propiedades anteriores aquellas que son object properties. Se almacenan en una lista.
    #Aquí se genera la consulta. Esta se hace para ver si la propiedad con la que estamos tratando es object property o no.
    sparql= SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery("""ASK WHERE {
    wdt:"""+prop+""" rdf:type owl:ObjectProperty
    }   
    """)
    sparql.setReturnFormat(XML)
    results = sparql.query().convert()
    string = results.toxml()
    
    #Se obtiene el valor booleano del xml string, siendo este "true" o "fals"
    es_OP = string[105:109]
    
    #En caso de que la variable es_OP sea "true", es decir la propiedad sea object propery:
    if es_OP == "true":
        
        #Aquí generamos la consulta. Se realiza esta consulta para obtener las clases que pertenecen a dicha propiedad.
        #Se realiza esta consulta para todas las object properties encontradas anteriormente.
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql.setQuery("""
        SELECT DISTINCT ?claseInicial ?claseInicialLabel ?prop ?propLabel ?Clase ?ClaseLabel
        WHERE {
        #occupation (P106) must be Formula 1 driver (Q10841764)
        ?claseInicial wdt:P31 wd:"""+clase+""" .
        ?claseInicial wdt:"""+prop +""" ?prop .
        ?prop wdt:P31 ?Clase .
        #to obtain labels in spanish:
        SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
        }  
        """)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        
        # Se añade a la lista "lista_clases_para_una_propiedad_completo" todas las clases que resultan de la consulta anterior. 
        lista_clases_para_una_propiedad_completo = []
        for result in results["results"]["bindings"]:
            lista_clases_para_una_propiedad_completo.append(result["Clase"]["value"])
        
        #Al tener estos valores en su inicio la URL "https:..", esto se elimina dejando solo el identificador de la clase.
        lista_clases_para_una_propiedad = []
        for ele in lista_clases_para_una_propiedad_completo:
            ele = ele[31:]
            lista_clases_para_una_propiedad.append(ele)
        
        #Ya tenemos la lista "lista_clases_para_una_propiedad" en la cual se encuentran todas las clases de la propiedad estudiada.
        
        #Se obtienen las value type constraint de las propiedades. La consulta siguiente las obtiene.
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql.setQuery("""SELECT ?Property_ ?Property_Label ?Property_Description ?class_ ?class_Label ?relation_ ?relation_Label
        WHERE {
        ?Property_ wikibase:directClaim wdt:"""+prop+""" .
        ?Property_ p:P2302 ?constraint_statement .
        ?constraint_statement ps:P2302 wd:Q21510865 .
        OPTIONAL {?constraint_statement pq:P2308 ?class_ .}
        OPTIONAL {?constraint_statement pq:P2309 ?relation_ .}

        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        } 
        """)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        #En la lista "lista_value_type_constraints_completa" se almacenan todas las value type constraint de la propiedad estudiada.
        lista_value_type_constraints_completa = []
        for result in results["results"]["bindings"]:
            lista_value_type_constraints_completa.append(result["class_"]["value"])
            
        #Al tener estos valores en su inicio la URL "https:..", esto se elimina dejando solo el identificador de la clase.
        lista_value_type_constraints = []
        for ele in lista_value_type_constraints_completa:
            ele = ele[31:]
            lista_value_type_constraints.append(ele)
        
        #Ya tenemos la lista "lista_value_type_constraints" con las value type constraint de la propiedad estudiada.
        if (lista_clases_para_una_propiedad != []) & (lista_value_type_constraints != []):
            #Se guardan en la lista "lista_clases_a_representar" aquellos identificadores que esten en ambas listas.
            lista_clases_a_representar = []
            for ele_VTC in lista_value_type_constraints:
                for ele_CPUP in lista_clases_para_una_propiedad:
                    if ele_VTC == ele_CPUP:
                        lista_clases_a_representar.append(ele_VTC)
            
            #Tenemos la lista "lista_clases_a_representar" con las clases que hay que representar para la propiedad estudiada.
            
            #Se eliminan los duplicados
            if lista_clases_a_representar != []:             
                result = []
                for ele in lista_clases_a_representar:
                    if ele not in result:
                        result.append(ele)
                        
                #Se muestran por pantalla los resultados: La propiedad a representar y sus respectivas clases.
                print("\n")
                print("Propiedad:", prop)
                print("Clases:", result)



Propiedad: P495
Clases: ['Q6256', 'Q7275', 'Q3024240']


Propiedad: P674
Clases: ['Q95074']


Propiedad: P840
Clases: ['Q82794', 'Q3895768']


Propiedad: P910
Clases: ['Q4167836']


Propiedad: P941
Clases: ['Q5']


Propiedad: P6338
Clases: ['Q5']


Propiedad: P50
Clases: ['Q5', 'Q43229']


Propiedad: P110
Clases: ['Q5']


Propiedad: P123
Clases: ['Q11032', 'Q2085381']


Propiedad: P136
Clases: ['Q483394']


Propiedad: P17
Clases: ['Q6256', 'Q43702', 'Q3624078']


Propiedad: P170
Clases: ['Q5']


Propiedad: P1434
Clases: ['Q559618']


Propiedad: P1080
Clases: ['Q559618']


Propiedad: P2354
Clases: ['Q13406463']


Propiedad: P166
Clases: ['Q618779', 'Q38033430']


Propiedad: P8345
Clases: ['Q196600']


Propiedad: P747
Clases: ['Q3331189']
